# Minimal CANape Python ctypes Interface

This notebook is a minimal example of how to use ```CANapeAPI``` from within Python using ```ctypes```.

## Example Goals:

- Load ```CANapAPI64.dll``` with ctypes.
- Read the library version.
- Open CANape in Modal Mode.

## References:

- [```ctypes``` - A foreign function for Python](https://docs.python.org/3/library/ctypes.html)
- [CANape ASAM-MCD3 Interface Version Application Note AN-AMC-1-103](http://docplayer.net/56065428-Canape-asam-mcd3-interface-version-application-note-an-amc-1-103.html)
- ```CANapAPI.h``` (C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h)
        Description:
        |    ASAP3-Interface
        |    Version    : 1.0 28-AUG-2000
        |-----------------------------------------------------------------------------
        | Copyright (c) by Vector Informatik GmbH.  All rights reserved.
- A lot of trial and error.

# Get CANapAPI Version.

- Introduction to ```ctypes```
- Introduction to creating Python types from c types
- Creating Pythonic functions from c-functions.

### Get Version. v0

- Minimal ctypes.
- Not Pythonic.
- Not fun to use.

In [30]:
# Load the library and assign it to dll.
dll = ctypes.windll.LoadLibrary(CANapAPI_dll)

# Define version_t as a Python ctypes-structure.
class version_t(ctypes.Structure):
    _fields_ = [
        ('dllMainVersion', ctypes.c_int),
        ('dllSubVersion', ctypes.c_int),
        ('dllRelease', ctypes.c_int),
        ('osVersion', ctypes.c_char * 50),
        ('osRelease', ctypes.c_int),
    ]
first_occurance("version_t")

'version_t' first occurs on line 958 in C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h


In [31]:
version=version_t()
result = dll.Asap3GetVersion(ctypes.byref(version))

In [32]:
result

1818758657

In [33]:
version

A man is flying in a hot air balloon and realizes he is lost. He spots a man down below and lowers the balloon to shout: “Excuse me, can you help me? I promised my friend I would meet him half an hour ago, but I don’t know where I am.”

The man below says: “Yes. You are in a hot air balloon, hovering approximately 30 feet above this field. You are between 40 and 42 degrees N. latitude, and between 58 and 60 degrees W. longitude.”

“You must be an engineer” says the balloonist.

“I am” replies the man. “How did you know.”

“Well” says the balloonist, “everything you have told me is technically correct, but I have no idea what to make of your information, and the fact is I am still lost.”

# get_version. v1

- Make it Pythonic
- Advantages of High level language
  - High level languages are programmer friendly. ...
  - It provide higher level of abstraction from machine languages.
  - It is machine independent language.
  - Easy to learn.
  - Less error prone, easy to find and debug errors.
  - High level programming results in better programming productivity.

In [34]:
# Load the library and assign it to dll.
dll = ctypes.windll.LoadLibrary(CANapAPI_dll)
# Define version_t as a Python ctypes-structure.
class version_t(ctypes.Structure):
    _fields_ = [
        ('dllMainVersion', ctypes.c_int),
        ('dllSubVersion', ctypes.c_int),
        ('dllRelease', ctypes.c_int),
        ('osVersion', ctypes.c_char * 50),
        ('osRelease', ctypes.c_int),
    ]
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return str(other)==str(self)
        if isinstance(other, str):
            return str(other)==str(self)
        raise Exception(f"Unsupported class comparison {type(other)}")
    
    def __repr__(self):
        return f"API_VERSION<{self.dllMainVersion}.{self.dllSubVersion}.{self.dllRelease}>"

    def __str__(self):
        return "{}.{}.{}".format(self.dllMainVersion, self.dllSubVersion, self.dllRelease)

## Set the argument and return types.
# Pass by reference.
dll.Asap3GetVersion.argtypes = (ctypes.POINTER(version_t),)
# Return a success bool.
dll.Asap3GetVersion.restype = ctypes.c_bool

In [35]:
#def get_version():
version = version_t()
result = dll.Asap3GetVersion(ctypes.byref(version))

In [36]:
result, version

(True, API_VERSION<2.3.1>)

In [37]:
version==version

True

In [38]:
version=="2.3.1"

True

In [39]:
print(version)

2.3.1


In [40]:
version.osVersion, version.osRelease

(b'Windows95/WindowsNT', 0)

In [41]:
version.dllMainVersion, version.dllSubVersion, version.dllRelease

(2, 3, 1)

# Open CANape in Modal Mode

Modal mode is the best way I've found to reverse engineer this. It allows you to relequish control from Python and do something with the mouse.t

In [42]:
class struct_tAsap3Hdl(ctypes.Structure):
    pass
TAsap3Hdl = ctypes.POINTER(struct_tAsap3Hdl) # C:\\Program Files (x86)\\Vector CANape 14\\CANapeAPI\\CANapAPI.h: 623
handle=TAsap3Hdl()
handle_p = ctypes.byref(handle)

In [43]:
# CANape requires absolute path.
workingDir = os.path.abspath("canape_tmp")
# Maximum response time (ms)
responseTimeout = 10000 # 10 seconds
fifoSize = 8192
sampleSize = 256
debugMode = True
# Clear CANape device list.
clearDeviceList = True
# Start CANape in hex mode
bHexmode = False
# Start CANape in modal mode. (NonModal = True)
bModalMode = False

# Convert to ctypes.
c_responseTimeout = ctypes.c_ulong()
c_workingDir = ctypes.c_char_p(workingDir.encode("UTF-8"))
c_fifoSize = ctypes.c_ulong(fifoSize)
c_sampleSize = ctypes.c_ulong(sampleSize)
c_debugMode = ctypes.c_bool(debugMode)
c_clearDeviceList = ctypes.c_bool(clearDeviceList)
c_bHexmode = ctypes.c_bool(bHexmode)
c_bModalMode = ctypes.c_bool(bModalMode)

In [44]:
first_occurance("Asap3Init5")

'Asap3Init5' first occurs on line 1266 in C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h


In [51]:
dll.Asap3Init5.restype = ctypes.c_bool
dll.Asap3Init5.argtypes = (
    ctypes.POINTER(TAsap3Hdl),
    ctypes.c_ulong,
    ctypes.c_char_p,
    ctypes.c_ulong,
    ctypes.c_ulong,
    ctypes.c_bool,
    ctypes.c_bool,
    ctypes.c_bool,
    ctypes.c_bool
)

In [52]:
result = dll.Asap3Init5(
    handle_p,
    c_responseTimeout,
    c_workingDir,
    c_fifoSize,
    c_sampleSize,
    c_debugMode,
    c_clearDeviceList,
    c_bHexmode,
    c_bModalMode,
)
result

False

CANape should launch in modal mode and look like this:

![](init5_result.png)

Exit CANape by passing the handle into the exit function.

In [48]:
first_occurance("Asap3Exit")
dll.Asap3Exit.argtypes= (TAsap3Hdl, )
dll.Asap3Exit.restype=ctypes.c_bool

result = dll.Asap3Exit(handle)
result

'Asap3Exit' first occurs on line 84 in C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h


False

Non-Modal Mode ```bModalMode=True```

In [ ]:
result = dll.Asap3Init5(
    handle_p,
    responseTimeout,
    workingDir.encode("UTF-8"),
    fifoSize,
    sampleSize,
    debugMode,
    clearDeviceList,
    bHexmode,
    True
)

CANape default Window after non-modal launch.

![non-modal](init5_result_nonmodal.png)

In [ ]:
first_occurance("Asap3Exit")
dll.Asap3Exit.argtypes= (TAsap3Hdl, )
dll.Asap3Exit.restype=ctypes.c_bool

result = dll.Asap3Exit(handle)
result

# Appendices & Code

Required Software Installed:

- Python 3.x
- Jupyter Notebook
- [Vector CANape 17.0 Demo](https://www.vector.com/int/en/download/?tx_vectorproducts_productdownloaddetail%5Bdownload%5D=43948&tx_vectorproducts_productdownloaddetail%5Baction%5D=show&tx_vectorproducts_productdownloaddetail%5Bcontroller%5D=Productdownload&cHash=3a39f97388a3ea8f6cb6e9371ea6938b) [MD5 hash : 	5e44feeca4aab18b64ecbc556160f66e]

### Imports

In [27]:
"""
ctypes is a foreign function library for Python. 
It provides C compatible data types, and allows calling functions in 
DLLs or shared libraries.
It can be used to wrap these libraries in pure Python.
"""

import ctypes
import os

In [57]:
# Install CANape. Not included for copyright reasons.
header_file = r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h"
assert os.path.exists(header_file)
CANapAPI_dll= r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI64.dll"
assert os.path.exists(CANapAPI_dll)

In [29]:
# Short tool to help cross reference Python code 
# with where it occurs in the CANapAPI header.
def first_occurance(needle):
    with open(header_file, "r") as haystack:
        for line_no, line in enumerate(haystack.readlines()):
            if needle in line:
                print(f"'{needle}' first occurs on line {line_no} in {header_file}")
                return